In [ ]:
from operator import itemgetter
from datetime import datetime

from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_fireworks.chat_models import ChatFireworks

In [ ]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
from langchain.agents import create_tool_calling_agent, create_react_agent, AgentExecutor
from langchain_community.agent_toolkits.load_tools import load_tools
from langchain_core.prompts import ChatPromptTemplate
from langchain.globals import set_debug, set_verbose
from langchain_core.output_parsers.string import StrOutputParser
from langchain import hub

set_debug(False)
set_verbose(False)

In [ ]:
import os
News_api_key = os.environ["NEWS_API_KEY"]

In [ ]:
TEMPERATURE = 0.0
MAX_NEW_TOKENS = 1000

MODELS = {
    "llama-v2-70b-chat": ChatFireworks(
        model_name="accounts/fireworks/models/llama-v2-70b-chat",
        temperature=TEMPERATURE,
        max_tokens=MAX_NEW_TOKENS,            
    ),
    "llama-v3-70b-instruct": ChatFireworks(
        model_name="accounts/fireworks/models/llama-v3-70b-instruct",
        temperature=TEMPERATURE,
        max_tokens=MAX_NEW_TOKENS,            
    ),    
    "firefunction": ChatFireworks(
        model_name="accounts/fireworks/models/firefunction-v1",
        temperature=TEMPERATURE,
        max_tokens=MAX_NEW_TOKENS,            
    ),
    "gpt3_5turbo": ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=TEMPERATURE,
        max_tokens=MAX_NEW_TOKENS,
    ),    
    "gpt4o": ChatOpenAI(
        model="gpt-4o",
        temperature=TEMPERATURE,
        max_tokens=MAX_NEW_TOKENS,
    ),
    "mistral_large": ChatMistralAI(
        model="mistral-large-latest",
        temperature=TEMPERATURE,
        max_tokens=MAX_NEW_TOKENS,
    ),
    "gemini_flash": ChatGoogleGenerativeAI(
        model="gemini-1.5-flash-latest",
        temperature=TEMPERATURE,
        max_tokens=MAX_NEW_TOKENS,
    ),
    "gemini_pro": ChatGoogleGenerativeAI(
        model="gemini-1.5-pro",
        temperature=TEMPERATURE,
        max_tokens=MAX_NEW_TOKENS,
    ),
}

model_type = "gemini_flash"
llm = MODELS[model_type]

parser = StrOutputParser()

In [ ]:
DATE = datetime.today().strftime('%Y-%m-%d')

llm_math_tool, weather_tool, news_tool, wikipedia_tool, arxiv_tool, websearch_tool = load_tools(["llm-math", "open-meteo-api", "news-api", "wikipedia", "arxiv", "ddg-search"], news_api_key=News_api_key, llm=llm)

#llm_math_tool.invoke("What is the square root of 4?")
#wikipedia_tool.invoke("How many people live in Prague?")
#print(weather_tool.invoke(prompt, verbose=False))
#print(news_tool.invoke("What are the 10 most important news in Prague? Answer in 10 bullet points"))
#print(arxiv_tool.invoke("List the title of 10 scientific papers about LLM agents published in this year.", verbose=True))
#print(websearch_tool.invoke("Who won the most Oscar in this year?"))
#print(pubmed_tool.invoke("List papers about Vortioxetin."))llm

In [ ]:
tools=[llm_math_tool, arxiv_tool, news_tool, weather_tool, wikipedia_tool, websearch_tool] # More tools could be added

if model_type.startswith("llama"):
    prompt_react = hub.pull("hwchase17/react")
    agent = create_react_agent(llm, tools, prompt_react)
else:
    prompt_tool_calling = ChatPromptTemplate.from_messages([
        ("system", "{system_prompt}"),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ])        
    agent = create_tool_calling_agent(llm, tools, prompt_tool_calling)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose = True, stream_runnable=False)
# , enable_automatic_function_calling=True

agent_chain = agent_executor | itemgetter("output")

In [ ]:
system_prompt = f"You're a helpful assistant. Always use tools to answer questions. Always use the Calculator for calculations, even when adding 2 numbers or calculating averages. The current date is {datetime.today().strftime('%Y-%m-%d')}"

#prompt = "What is the square root of 2?"
#prompt = "What is the weather like in Budapest?"
#prompt = "What are the 10 trending news in Budapest Hungary? Answer in 10 bullet points."
#prompt = "How many people live in Budapest?"
#prompt = "Who won the most Oscar in this year?"
#prompt = "List the title of 10 scientific papers about LLM agents published in this year."

#prompt = "What are the maximum, minimum and average temperature values as well as the sum of precipitations on each of the coming 3 days in Budapest?"
#prompt = f"Create a table that contains the date as well as the maximum, minimum and average temperature values as well as the sum of precipitations in the coming 7 days in Budapest"
prompt = f"""Create a table that contains the date as well as the maximum, minimum and average temperature values as well as the sum of precipitations separately for each of the coming 7 days in Budapest.
Then include a line with each of the average of the maximum, minimum and average temperature values for these 7 days. And after all that also write a list with 10 current news in Budapest Hungary."""

prompt = "What is the square root of 2?"
streaming = False

response = ""
if streaming == True:
    for new_token in agent_chain.stream({"system_prompt": system_prompt, "input": prompt}):
        print(new_token, end="")
        response = response + new_token
else:
    response = agent_chain.invoke({"system_prompt": system_prompt, "input": prompt})
    print(response)
print("")

In [ ]:
#agent_executor.invoke({"input": "How many people live in Budapest and Prague together?"})
print(agent_chain.invoke({"system_prompt": system_prompt, "input": "What is the square root of 2?"}))

In [ ]:
print(agent_chain.invoke({"system_prompt": system_prompt, "input": "What is the weather like in Budapest?"}))

In [ ]:
print(agent_chain.invoke({"system_prompt": system_prompt, "input": "What are the 10 trending news in Budapest Hungary? Answer in 10 bullet points."}))

In [ ]:
print(agent_chain.invoke({"system_prompt": system_prompt, "input": "How many people live in Budapest?"}))

In [ ]:
print(agent_chain.invoke({"system_prompt": system_prompt, "input": "Who won the most Oscar in this year?"}))

In [ ]:
print(agent_chain.invoke({"system_prompt": system_prompt, "input": "List the title of 10 scientific papers about LLM agents published in this year."}))

In [ ]:
print(agent_chain.invoke({"system_prompt": system_prompt, "input": "What are the maximum, minimum and average temperature values as well as the sum of precipitations on each of the coming 3 days in Budapest?"}))

In [ ]:
print(agent_chain.invoke({"system_prompt": system_prompt, "input": "Create a table that contains the date as well as the maximum, minimum and average temperature values as well as the sum of precipitations in the coming 7 days in Budapest?"}))

In [ ]:
print(agent_chain.invoke({"system_prompt": system_prompt, "input": """Create a table that contains the date as well as the maximum, minimum and average temperature values as well as the sum of precipitations separately for each of the coming 7 days in Budapest.
Then include a line with each of the average of the maximum, minimum and avarage temperature values for these 7 days. And after all that also write a list with 10 current news in Budapest Hungary."""}))